### Import Data

In [34]:
import pandas as pd
import numpy as np


# string processing
import re

from keras.utils.np_utils import to_categorical

from sklearn import feature_extraction, model_selection, naive_bayes, pipeline, manifold, preprocessing, feature_selection, metrics

In [35]:
# to navigate to the data location
import os

# get current directory 
path = os.getcwd() 

# parent directory
parent = os.path.dirname(path)

df_merge_quality = pd.read_csv(parent + '/data/US_patent_abstract_5000_2015_with_title_1_5y.csv')
df_merge_quality.shape

(5000, 30)

In [36]:
df = df_merge_quality[['claims_text', 'quality_rank']]
df

,claims_text,quality_rank
0,What is claimed is: \n \n 1. An invi...,0
1,What is claimed is: \n \n 1. An impl...,0
2,What is claimed is: \n \n 1. A spear...,1
3,1. A computer-implemented method for the autom...,1
4,What is claimed is: \n \n 1. A semic...,0
...,...,...
4995,What is claimed is: \n \n 1. A compu...,1
4996,What is claimed is: \n \n 1. A displ...,1
4997,What is claimed is: \n \n 1. A compo...,1
4998,The invention claimed is: \n \n 1. A...,1


In [37]:
df['claims_text'].iloc[0]

'What is claimed is: \n     \n       1. An invitation information push method comprising:\n after receiving an invitation request sent by a microblog user, a server sending invitation information to clients corresponding to a number of invited users carried in the invitation request, wherein the invited users are users who have not registered microblog, and the number of the invited users N is greater than or equal to 1; and \n upon receiving the invitation information, each client creating an invitation information guide to guide the user who has not registered microblog to register microblog; \n wherein the server sending the invitation information to the clients corresponding to the invited users comprises: \n determining, by the server, whether one or more of the invited users carried in the invitation request are in a restricted list; and \n if none of the invited users carried in the invitation request is in the restricted list, sending the invitation information to the clients c

In [38]:
df['claims_text'] = df['claims_text'].apply(str)

/home/lin_menghsien/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


### Create a model

In [39]:
import numpy as np
import pandas as pd
import pickle as cPickle
from collections import defaultdict
import re

from bs4 import BeautifulSoup

import sys
import os
import tensorflow as tf
from tensorflow import keras

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical

from keras.layers import Embedding
from keras.layers import Dense, Input, Flatten
from keras.layers import Conv1D, MaxPooling1D, Embedding, Dropout, LSTM, GRU, Bidirectional, TimeDistributed
from keras.models import Model

from keras import backend as K
from keras.engine.topology import Layer, InputSpec
from keras import initializers

In [40]:
# more CNN library
import os
import sys
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.layers import Dense, Input, GlobalMaxPooling1D
from keras.layers import Conv1D, MaxPooling1D, Embedding
from keras.models import Model
from keras.initializers import Constant


In [41]:
MAX_SEQUENCE_LENGTH = 1500  # [Steven] I added for the CNN, to take only the 1500 words.
# MAX_SENT_LENGTH = 100
# MAX_SENTS = 15
MAX_NB_WORDS = 30000  # [Steven] this should be number of unique word / vocabulary
EMBEDDING_DIM = 100
VALIDATION_SPLIT = 0.2

In [42]:
def clean_str(string):
    """
    Tokenization/string cleaning for dataset
    Every dataset is lower cased except
    """
    string = re.sub(r"\\", "", str(string))    
    string = re.sub(r"\'", "", str(string))    
    string = re.sub(r"\"", "", str(string))    
    return string.strip().lower()

In [43]:

data_train = df
data_train

,claims_text,quality_rank
0,What is claimed is: \n \n 1. An invi...,0
1,What is claimed is: \n \n 1. An impl...,0
2,What is claimed is: \n \n 1. A spear...,1
3,1. A computer-implemented method for the autom...,1
4,What is claimed is: \n \n 1. A semic...,0
...,...,...
4995,What is claimed is: \n \n 1. A compu...,1
4996,What is claimed is: \n \n 1. A displ...,1
4997,What is claimed is: \n \n 1. A compo...,1
4998,The invention claimed is: \n \n 1. A...,1


In [44]:
data_train.shape

(5000, 2)

In [45]:
import nltk
from nltk import tokenize

abstracts = [] # abstracts is list of list of list to hold each sentences of each abstract (the most complete data)
labels = [] # label is just a list holding our label which is quality_index
texts = []  # texts to hold each complete abstract as list of list (note: abstract not breaking up to sentence level)
for idx in range(data_train.claims_text.shape[0]): # for each row
    #text = BeautifulSoup(data_train.text[idx])
    #print(clean_str(str(data_train.iloc[idx]['text'])))
    text = clean_str(str(data_train.iloc[idx]['claims_text'])) # text is each complete abstract
    texts.append(text) # texts to hold each complete abstract as list of string (note: abstract not breaking up to sentence level)
    sentences = tokenize.sent_tokenize(text) # sentences is list of string holding each complete sentence of one abstract (but it's just an intermediate variable, not used directly in later code)
    abstracts.append(sentences) # abstracts is list of list of string to hold each sentences of each abstract (the most complete data) (this is what we use )
    labels.append(data_train.iloc[idx]['quality_rank']) # label is just a list holding our label which is quality_index

In [46]:
tokenizer = Tokenizer(num_words=MAX_NB_WORDS) # intend to use next line .fit_on_texts to index each word within specific abstract at current iteration/loop, the more frequent word has lower index number, it is a dictionary format, it's like a unique vocabulary index
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)
# data = np.zeros((len(texts), MAX_SENTS, MAX_SENT_LENGTH), dtype='int32')

In [47]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 31084 unique tokens.


In [48]:
len(sequences)

5000

In [49]:
# create array of equal length to feed into model
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post', truncating='post') # The default is pre-padding, should I try post-padding ? => padding='post', truncating='post'. I think it has to do with whether the beginning words are more important, or later words more important. You may want to try both approaches and compare result?
data

array([[216,   7,  78, ...,   0,   0,   0],
       [216,   7,  78, ...,   0,   0,   0],
       [216,   7,  78, ...,   0,   0,   0],
       ...,
       [216,   7,  78, ...,   0,   0,   0],
       [  1, 430,  78, ..., 274,   9,   1],
       [216,   7,  78, ...,   0,   0,   0]], dtype=int32)

In [50]:
data.shape

(5000, 1500)

In [51]:
labels = to_categorical(np.asarray(labels))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', labels.shape)

Shape of data tensor: (5000, 1500)
Shape of label tensor: (5000, 2)


In [52]:

num_validation_samples = int(VALIDATION_SPLIT * data.shape[0])
num_validation_samples

1000

In [53]:
x_train = data[:-num_validation_samples]
y_train = labels[:-num_validation_samples]
x_val = data[-num_validation_samples:]
y_val = labels[-num_validation_samples:]

print('Number of positive and negative reviews in traing and validation set')
print(y_train.sum(axis=0))
print(y_val.sum(axis=0))


Number of positive and negative reviews in traing and validation set
[2397. 1603.]
[595. 405.]


In [54]:
y_train

array([[1., 0.],
       [1., 0.],
       [0., 1.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [55]:
y_val

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [0., 1.],
       [0., 1.],
       [0., 1.]], dtype=float32)

In [56]:
# to navigate to the data location
import os

# get current directory 
path = os.getcwd() 

# parent directory
parent = os.path.dirname(path)

In [57]:
# import GloVe word embedding
GLOVE_DIR = "" 
embeddings_index = {}
f = open(os.path.join(parent, GLOVE_DIR, 'glove.6B.100d.txt'))
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [58]:
print('Total %s word vectors.' % len(embeddings_index))

Total 400000 word vectors.


In [59]:
len(word_index) + 1

31085

In [60]:
# prepare embedding matrix
num_words = min(MAX_NB_WORDS, len(word_index) + 1)
embedding_matrix = np.zeros((num_words, EMBEDDING_DIM))  # EMBEDDING_DIM = 100 since we import 100d GloVe
for word, i in word_index.items():
    if i >= MAX_NB_WORDS:
        continue
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [61]:
# load pre-trained word embeddings into an Embedding layer
# note that we set trainable = False so as to keep the embeddings fixed
embedding_layer = Embedding(num_words,
                            EMBEDDING_DIM,
                            embeddings_initializer=Constant(embedding_matrix),
                            input_length=MAX_SEQUENCE_LENGTH,
                            trainable=False)

In [62]:
import os, sys, re, json, time, datetime, shutil
from numpy.random import seed
seed(5)
import tensorflow
tensorflow.random.set_seed(5)

In [63]:
# Parameter setting
epochs = 10
embed_dim = EMBEDDING_DIM  # already specify with EMBEDDING_DIM variable above
dense_layer_dims = [100]
dropout_rate = 0.2
num_filters = [25, 25, 50, 50, 50]
kernel_sizes = [3, 4, 15, 50, 150]
num_classes = labels.shape[1] # 

# Model building
start_time = time.time()
sequence_input = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
h = embedding_layer(sequence_input) # remember embedding_layer assign with GloVe's weight

conv_layers_for_all_kernel_sizes = []
counter = 0
for kernel_size, filters in zip(kernel_sizes, num_filters): 
    conv_layer = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, activation='relu')(h)
    conv_layer = keras.layers.GlobalMaxPooling1D()(conv_layer)
    conv_layers_for_all_kernel_sizes.append(conv_layer)
    counter += 1
    #print(conv_layers_for_all_kernel_sizes)
if counter == 1: # need this since layers.concatenate throw error when there is only 1 item in conv_layers_for_all_kernel_sizes
    h = conv_layer
else:
    h = keras.layers.concatenate(conv_layers_for_all_kernel_sizes, axis=1) # Concat the feature maps from each different size.

    h = keras.layers.Dropout(rate=dropout_rate)(h) # Dropout can help with overfitting (improve generalization) by randomly 0-ing different subsets of values in the vector

for dim in dense_layer_dims:
    h = keras.layers.Dense(dim, activation = 'relu')(h) # [Steven] I believe this add additional fully-connected hidden layer with the hope to increase model accuracy

prediction = keras.layers.Dense(num_classes, activation='sigmoid')(h)

model = keras.Model(inputs=sequence_input, outputs=prediction)
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # From information theory notebooks.
              metrics=['accuracy'])        # What metric to output as we train.



In [64]:
# setup checkpoint

checkpoint_path = "ckpt_CNN_US_5years/"

ckpt = tf.train.Checkpoint(model = model) 

ckpt_manager = tf.train.CheckpointManager(ckpt, checkpoint_path, max_to_keep=2)

if ckpt_manager.latest_checkpoint:
    ckpt.restore(ckpt_manager.latest_checkpoint)
    print("Latest checkpoint restored!!")

Latest checkpoint restored!!


In [65]:
class MyCustomCallback(tf.keras.callbacks.Callback):

    def on_epoch_end(self, epoch, logs=None):
        ckpt_manager.save()
        print("Checkpoint saved at {}.".format(checkpoint_path))

In [33]:
# train
model.reset_states()
model.fit(x_train, 
          y_train, 
          epochs=epochs, 
          validation_data=(x_val, y_val),
         callbacks=[MyCustomCallback()])


Epoch 1/10
125/125 [==============================] - 346s 3s/step - loss: 0.7282 - accuracy: 0.5913 - val_loss: 0.6452 - val_accuracy: 0.6290
Epoch 2/10
125/125 [==============================] - 335s 3s/step - loss: 0.6242 - accuracy: 0.6585 - val_loss: 0.6611 - val_accuracy: 0.6220
Epoch 3/10
125/125 [==============================] - 337s 3s/step - loss: 0.5459 - accuracy: 0.7253 - val_loss: 0.6304 - val_accuracy: 0.6560
Epoch 4/10
125/125 [==============================] - 341s 3s/step - loss: 0.4086 - accuracy: 0.8230 - val_loss: 0.8241 - val_accuracy: 0.5570
Epoch 5/10
  5/125 [>.............................] - ETA: 4:01 - loss: 0.3118 - accuracy: 0.8375

KeyboardInterrupt: 

In [66]:
# disconnected, re-start from epoch = 5

# train
model.fit(x_train, 
          y_train, 
          epochs=6, 
          validation_data=(x_val, y_val),
         callbacks=[MyCustomCallback()])


Epoch 1/6
125/125 [==============================] - 319s 3s/step - loss: 0.0599 - accuracy: 0.9815 - val_loss: 1.0406 - val_accuracy: 0.6430
Epoch 2/6
125/125 [==============================] - 321s 3s/step - loss: 0.0352 - accuracy: 0.9915 - val_loss: 1.2343 - val_accuracy: 0.6240
Epoch 3/6
125/125 [==============================] - 321s 3s/step - loss: 0.0455 - accuracy: 0.9868 - val_loss: 1.1320 - val_accuracy: 0.6250
Epoch 4/6
125/125 [==============================] - 321s 3s/step - loss: 0.0329 - accuracy: 0.9893 - val_loss: 1.2046 - val_accuracy: 0.6360
Epoch 5/6
125/125 [==============================] - 321s 3s/step - loss: 0.0517 - accuracy: 0.9827 - val_loss: 1.4911 - val_accuracy: 0.5810
Epoch 6/6
125/125 [==============================] - 321s 3s/step - loss: 0.1067 - accuracy: 0.9635 - val_loss: 1.2202 - val_accuracy: 0.6250


In [67]:
pred_test = model.predict(x_val)
pred_test

array([[9.8661357e-01, 1.2649804e-02],
       [5.7794249e-01, 4.7276512e-01],
       [9.4323725e-01, 6.8224579e-02],
       ...,
       [3.2758713e-04, 9.9980068e-01],
       [1.1646181e-02, 9.8710781e-01],
       [9.7226346e-01, 3.0034065e-02]], dtype=float32)

In [68]:
np.savetxt('Predict_Output/CNN_5yr_claims_dev_prob.csv', pred_test)

In [69]:
predicted = [np.argmax(pred) for pred in 
             pred_test]

In [70]:
y_test_binary = df['quality_rank'][4000:].values
y_test_binary

array([0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0,

In [71]:
## Accuracy, Precision, Recall
accuracy = metrics.accuracy_score(y_test_binary, predicted)
auc = metrics.roc_auc_score(y_test_binary, predicted)  # predicted_prob), check doc, seems the second argument required to be shape (n_samples,) for binary case 
                            #multi_class="ovr") # check documentation and seems "ovr" not good for only binary target class
print("Accuracy:",  round(accuracy,3))
print("Auc:", round(auc,3))
print("Detail:")
print(metrics.classification_report(y_test_binary, predicted))



Accuracy: 0.625
Auc: 0.592
Detail:
              precision    recall  f1-score   support

           0       0.66      0.77      0.71       595
           1       0.55      0.42      0.47       405

    accuracy                           0.62      1000
   macro avg       0.60      0.59      0.59      1000
weighted avg       0.61      0.62      0.61      1000

